In [1]:
import math
import time
import timeit
import numpy as np
import anguilla.hypervolume as hv
#from anguilla.util import random_cliff_3d

In [2]:
def random_cliff_3d(n, rng=np.random.default_rng()):
    vs = rng.standard_normal(size=(n, 2))
    ys = np.zeros((n, 3))
    for i in range(n):
        c = np.linalg.norm(vs[i])
        ys[i, 0:2] = 10.0 * np.abs(vs[i]) / c
    ys[:, 2] = rng.uniform(low=0.0, high=10.0, size=n)
    return ys

In [3]:
import benchit
benchit.setparams(environ='notebook')

Notebook environment set! Use "fontsize" & "figsize" args with plot method for better viewing experience.


In [4]:
datasets = {}
for n in np.arange(100, 1100, 100):
    front = random_cliff_3d(n)
    nadir = np.max(front, axis=0) + 1e-4
    datasets[n] = (front, nadir)

In [5]:
datasets_b = {}
for n in np.arange(5, 100, 5):
    front = random_cliff_3d(n)
    nadir = np.max(front, axis=0) + 1e-4
    datasets_b[n] = (front, nadir)

In [6]:
def shark_bindings(*args, **kwargs):
    return hv.shark_calculate(*args, **kwargs)

def anguilla_btree(*args, **kwargs):
    return hv.calculate(*args, use_btree=True, **kwargs)

def anguilla_rbtree(*args, **kwargs):
    return hv.calculate(*args, use_btree=False, **kwargs)

implementations = [shark_bindings, anguilla_btree, anguilla_rbtree]

In [ ]:
t = benchit.timings(implementations, datasets, multivar=True, input_name='Front size')

Loop datasets :  50%|█████     | 5/10 [00:48<00:48,  9.73s/it]

In [ ]:
_ = t.plot()

In [ ]:
t.speedups(anguilla_rbtree)

In [ ]:
t_b = benchit.timings(implementations, datasets_b, multivar=True, input_name='Front size')

In [ ]:
_ = t_b.plot()

In [ ]:
t_b.speedups(anguilla_rbtree)

In [ ]:
def shark_bindings_c(*args, **kwargs):
    return hv.shark_contributions(*args, **kwargs)

def anguilla_btree_c(*args, **kwargs):
    return hv.contributions(*args, use_btree=True, **kwargs)

def anguilla_rbtree_c(*args, **kwargs):
    return hv.contributions(*args, use_btree=False, **kwargs)

implementations_c = [shark_bindings_c, anguilla_btree_c, anguilla_rbtree_c]

In [ ]:
t_c = benchit.timings(implementations_c, datasets, multivar=True, input_name='Front size')

In [ ]:
_ = t_c.plot()

In [ ]:
t_c.speedups(anguilla_rbtree_c)

In [ ]:
t_b_c = benchit.timings(implementations_c, datasets_b, multivar=True, input_name='Front size')

In [ ]:
_ = t_b_c.plot()

In [ ]:
t_b_c.speedups(anguilla_rbtree_c)

In [ ]:
def random_3d_front(n, duplicates=False, dominated=False):
    rng = np.random.default_rng()
    cliff3d = random_cliff_3d(n, rng=rng)
    # Add some repeated entries.
    dups = None
    if duplicates:
        dups = cliff3d[rng.choice(n, max(2, n // 3), replace=False), :]
    dom = None
    if dominated:
        m = max(2, n // 3)
        dom = cliff3d[rng.choice(n, m, replace=False), :]
        for i in range(m):
            dom[i, rng.integers(0, 2)] += 0.01
    front = np.vstack([x for x in [cliff3d, dups, dom] if x is not None])
    rng.shuffle(front)
    nadir = np.ceil(np.max(front, axis=0))
    return front, nadir

In [ ]:
def random_2d_3d_front(n, duplicates=False, dominated=False):
    front_3d, nadir_3d = random_3d_front(n, duplicates=duplicates, dominated=dominated)
    n = len(front_3d)
    front_3d[:, 2] = 0.0
    nadir_3d[2] = 1.0
    front_2d = np.delete(front_3d, 2, 1)
    nadir_2d = np.empty(2)
    nadir_2d[0] = nadir_3d[0]
    nadir_2d[1] = nadir_3d[1]
    return front_3d, nadir_3d, front_2d, nadir_2d

In [ ]:
front_3d, nadir_3d, front_2d, nadir_2d = random_2d_3d_front(10, dominated=False)

In [ ]:
for x, y, z in front_3d:
    print(x, y, z)

In [ ]:
print(nadir_3d)

In [ ]:
hv.contributions(front_2d, nadir_2d, non_dominated=False)

In [ ]:
hv.contributions(front_3d, nadir_3d)

In [ ]:
hv.shark_contributions(front_3d, nadir_3d)

In [ ]:
hv.shark_contributions(front_2d, nadir_2d)

In [ ]:
import pygmo as pg
hvinstance = pg.hypervolume(front_3d)
hvinstance.contributions(ref_point=nadir_3d)